In [222]:
# 导入相关包
from bs4 import BeautifulSoup
import requests
import base64
import re
import json
import pandas as pd
import sys
import pkuseg
pd.set_option('max_columns',10)
pd.set_option('max_row',None)

In [223]:
headers1 = {
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'zh-CN,zh;q=0.9,en;q=0.8',
    'cache-control': 'max-age=0',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.132 Safari/537.36'
}
headers2 = {
    'Accept': '*/*',
    'Accept-Encoding': 'gzip, deflate',
    'Accept-Language': 'zh-CN,zh;q=0.9,en;q=0.8',
    'Connection': 'keep-alive',
    'Host': 'comment.sina.com.cn',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.132 Safari/537.36'
}
url = 'https://news.sina.com.cn/'
rsp = requests.get(url,headers=headers1)
rsp.encoding = 'utf-8'
soup = BeautifulSoup(rsp.text,'lxml')
news_urls = []
for i in soup.find_all(href=re.compile(r'https://news.sina.com.cn/[a-zA-Z]/\d{4}-\d{2}-\d{2}/doc-[a-z]{8}[0-9]{7}.shtml')):
    news_urls.append(i['href'])

---

In [224]:
cmnt_url = 'http://comment.sina.com.cn/page/info?version=1&format=json&channel=gn&newsid=comos-{}&group=0&compress=0&ie=utf-8&oe=utf-8&page={}&page_size=10&t_size=3&h_size=3&thread=1&uid=7326475476&callback=jsonp_{}&_={}'
cmnt_pt = re.compile(r'(.*doc-i)([a-z]{7}\d{7})(.shtml)')


In [225]:

cookies=''
callback='1584187877416'

total_cmnt = []
total_area = []
total_agree = []
total_name = []
total_title = []

for news in news_urls: 
    area = []
    page = 1
    content = []
    agree = []
    nickname = []
    title = []
    reply_num = 0
    news_id = re.match(cmnt_pt,news).group(2)
    for page in range(1,20):
        try:
            new_url = cmnt_url.format(news_id,str(page),callback,callback)
            r = requests.get(new_url,headers=headers2,cookies=cookies)
            cookies = r.cookies
            r.encoding = 'utf-8'
            r.text.encode('utf-8').decode('unicode_escape')
            data = re.match(r'(.*)\((.*)\)',r.text).group(2)
            j = json.loads(data)
        except BaseException as e:
            print("ERROR: ")
            print(e)
        else:
            if j['result']['status']['code'] != 0:
                break
            cmnt_num = j['result']['count']['show']    #  总评论数
            for i in range(len(j['result']['cmntlist'])):
                area.append(j['result']['cmntlist'][i]['area'])
                content.append(j['result']['cmntlist'][i]['content'])
                reply_num += int(j['result']['cmntlist'][i]['content_ext']['reply'])
                agree.append(j['result']['cmntlist'][i]['agree'])
                nickname.append(j['result']['cmntlist'][i]['nick'])
                title.append(j['result']['news']['title'])
            if reply_num + len(content) >= cmnt_num:
                break
    total_area.extend(area)
    total_agree.extend(agree)
    total_cmnt.extend(content)
    total_name.extend(nickname)
    total_title.extend(title)

ERROR: 
Expecting value: line 1 column 1 (char 0)
ERROR: 
Expecting value: line 1 column 1 (char 0)


In [226]:
title_index = list(set(total_title))
title_index.sort(key=total_title.index)

In [227]:
comment_dict = {'新闻标题':total_title,'所在地':total_area,'评论':total_cmnt,'用户ID':total_name,'点赞数':total_agree}
comment_df = pd.DataFrame(data=comment_dict)
comment_df.to_csv('新浪新闻评论.csv', sep=',', encoding='utf-8',index=False)
df = pd.read_csv('新浪新闻评论.csv')
df = df.groupby('新闻标题').apply(lambda x:x.sort_values('点赞数',ascending=False)).reset_index(drop=True)
df.to_csv('新浪新闻评论.csv', sep=',', encoding='utf-8',index=False)
df = df.set_index(['新闻标题','点赞数'])
df

所在地  \
新闻标题                                  点赞数                  
98岁天文学泰斗韩天芑及夫人新冠肺炎治愈出院                28     新疆博尔塔拉蒙古自治州   
                                      15            浙江杭州   
                                      14              浙江   
                                      7               天津   
                                      6             浙江宁波   
                                      4               上海   
                                      2               重庆   
                                      1             山西大同   
                                      1             四川自贡   
                                      1             河北唐山   
                                      0             辽宁朝阳   
                                      0           永安生命公园   
                                      0             广东东莞   
                                      0             江苏南京   
“郑州毒王”同事一家三口在周口居家隔离                   33            浙江杭州   
                                      17              北京   
                                      12              北京   
                                      5             四川成都   
                                      0             湖北武汉   
                                      0             河北廊坊   
上海延长季节性暂停活禽活交易期限                      0             江苏泰州   
上海浦东机场的这张照片，刷屏了！                      68             新加坡   
                                      49            浙江杭州   
                                      43              上海   
                                      20            浙江杭州   
                                      18            宁夏银川   
                                      15              山东   
                                      12            河南郑州   
                                      11              上海   
                                      9               北京   
                                      9             湖北武汉   
                                      9             广东深圳   
                                      6             湖南长沙   
                                      4               北京   
                                      2               上海   
                                      2               浙江   
                                      2             辽宁沈阳   
                                      2               北京   
                                      2               北京   
                                      1             浙江杭州   
                                      1             湖北武汉   
                                      1               上海   
                                      1               北京   
                                      1               北京   
                                      1             北京海淀   
                                      0          内蒙古呼和浩特   
                                      0             江苏徐州   
与疫情赛跑！中国专家组在意大利开工首日全记录                23              重庆   
                                      6             山东潍坊   
                                      5             四川成都   
                                      3             广东深圳   
                                      3             江苏南京   
                                      2               北京   
                                      0               重庆   
                                      0             北京海淀   
                                      0             四川达州   
                                      0             四川成都   
中国科大附一院两名专家赴伊朗协助疫情防控                  2             广东佛山   
                                      2             山东菏泽   
                                      1             安徽宣城   
                                      1     云南红河哈尼族彝族自治州   
                                      1        甘肃临夏回族自治州   
中央指导组派员处理王烁同志因公殉职善后事宜并向家属慰问           398           湖北黄冈   
                                      306           陕西安康   
                                      241           广东广州   
                                 

In [228]:
# seg = pkuseg.pkuseg()
# for i in total_area:
#     text = seg.cut(i)
#     print(text)